In [2]:
import pandas as pd
import numpy as np
import PIL as Image
import os
import cv2

In [17]:
# Load the npy images and masks
RAW_IMAGES = '/media/network/hdd/public_datasets_archive/01_notebooks/pannuke/raw_files4inference/breast/raw_images/images.npy'
RAW_MASKS = '/media/network/hdd/public_datasets_archive/01_notebooks/pannuke/raw_files4inference/breast/masks/masks.npy'

In [18]:
OUTPUT_BASE_DIR = '/home/KutumLabGPU/Documents/santosh/TNBC-project/input-dir/pannuke/breast'

if not os.path.exists(OUTPUT_BASE_DIR):
    os.makedirs(OUTPUT_BASE_DIR)

OUTPUT_PATCHES_DIR = os.path.join(OUTPUT_BASE_DIR, 'patches')
OUTPUT_MASKS_DIR = os.path.join(OUTPUT_BASE_DIR, 'masks')

if not os.path.exists(OUTPUT_PATCHES_DIR):
    os.makedirs(OUTPUT_PATCHES_DIR)
    
if not os.path.exists(OUTPUT_MASKS_DIR):
    os.makedirs(OUTPUT_MASKS_DIR)

In [19]:
TARGET_PATCH_SIZE = (1024, 1024)

In [20]:
print(f"Loading images from: {RAW_IMAGES}")
images = np.load(RAW_IMAGES, allow_pickle=True)
print(f"Loaded images shape: {images.shape}")

Loading images from: /media/network/hdd/public_datasets_archive/01_notebooks/pannuke/raw_files4inference/breast/raw_images/images.npy
Loaded images shape: (10, 256, 256, 3)


In [21]:
print(f"Loading masks from: {RAW_MASKS}")
masks = np.load(RAW_MASKS, allow_pickle=True)
print(f"Loaded masks shape: {masks.shape}")

Loading masks from: /media/network/hdd/public_datasets_archive/01_notebooks/pannuke/raw_files4inference/breast/masks/masks.npy
Loaded masks shape: (10, 256, 256, 6)


In [22]:
def upscale_and_save_as_png(data_array: np.ndarray, output_directory: str, target_size: tuple, is_mask: bool = False):
    """
    Upscales a batch of images or masks to a target size and saves them as individual .png files.

    Args:
        data_array (np.ndarray): The input NumPy array containing images or masks.
                                 Expected shape: (num_items, height, width, [channels])
        output_directory (str): The path to the directory where the upscaled files will be saved.
        target_size (tuple): A tuple (width, height) representing the desired output resolution.
        is_mask (bool): If True, uses cv2.INTER_NEAREST interpolation (for masks) and converts to uint8.
                        If False, uses cv2.INTER_LINEAR interpolation (for images) and converts to uint8.
    """
    interpolation_method = cv2.INTER_NEAREST if is_mask else cv2.INTER_LINEAR
    file_prefix = "mask" if is_mask else "image"
    total_items = data_array.shape[0]

    print(f"Starting upscaling and saving for {total_items} {file_prefix}s as PNGs to {output_directory}...")

    for i in range(total_items):
        original_data = data_array[i]

        if isinstance(original_data, (list, tuple)): 
            original_data = np.array(original_data)
        
        if original_data.dtype != np.float32:
            original_data = original_data.astype(np.float32)

        upscaled_data = None
        if is_mask:
            if original_data.ndim == 3 and original_data.shape[2] > 1:
            
                single_channel_mask = np.argmax(original_data, axis=-1)
                upscaled_data = cv2.resize(
                    single_channel_mask,
                    target_size,
                    interpolation=interpolation_method
                )
            else:
                upscaled_data = cv2.resize(
                    original_data,
                    target_size,
                    interpolation=interpolation_method
                )
            
            upscaled_data = np.clip(upscaled_data, 0, 255).astype(np.uint8) 
        else: 
            upscaled_data = cv2.resize(
                original_data, 
                target_size,
                interpolation=interpolation_method
            )
            min_val = upscaled_data.min()
            max_val = upscaled_data.max()
            if max_val == min_val: # Handle cases of uniform images (e.g., completely black or white)
                upscaled_data = np.full_like(upscaled_data, 255 if max_val > 0 else 0, dtype=np.uint8)
            else:
                upscaled_data = ((upscaled_data - min_val) / (max_val - min_val) * 255).astype(np.uint8)

        output_filepath = os.path.join(output_directory, f"{file_prefix}_{i+1:02d}.png")
        cv2.imwrite(output_filepath, upscaled_data)

        if (i + 1) % 100 == 0 or (i + 1) == total_items:
            print(f"  Processed and saved {i + 1}/{total_items} {file_prefix}s as PNGs.")


In [23]:
print("\n--- Upscaling and Saving Images as PNGs ---")
upscale_and_save_as_png(images, OUTPUT_PATCHES_DIR, TARGET_PATCH_SIZE, is_mask=False)


--- Upscaling and Saving Images as PNGs ---
Starting upscaling and saving for 10 images as PNGs to /home/KutumLabGPU/Documents/santosh/TNBC-project/input-dir/pannuke/breast/patches...
  Processed and saved 10/10 images as PNGs.


In [24]:
print("\n--- Upscaling and Saving Masks as PNGs ---")
upscale_and_save_as_png(masks, OUTPUT_MASKS_DIR, TARGET_PATCH_SIZE, is_mask=True)

print("\n--- Upscaling and saving process complete! ---")


--- Upscaling and Saving Masks as PNGs ---
Starting upscaling and saving for 10 masks as PNGs to /home/KutumLabGPU/Documents/santosh/TNBC-project/input-dir/pannuke/breast/masks...
  Processed and saved 10/10 masks as PNGs.

--- Upscaling and saving process complete! ---
